In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import StratifiedKFold

In [3]:
train_with_spray = pd.read_csv('/Users/nichollejames/Desktop/mosquitos/kaggle_competition/clean_data/train_with_spray.csv')
train_msq = pd.read_csv('/Users/nichollejames/Desktop/mosquitos/kaggle_competition/clean_data/train_msq.csv')

In [4]:
train_msq.columns.tolist()

['Unnamed: 0',
 'Trap',
 'Date',
 'NumMosquitos_x',
 'Address',
 'Species',
 'Block',
 'Street',
 'AddressNumberAndStreet',
 'Latitude',
 'Longitude',
 'AddressAccuracy',
 'NumMosquitos_y',
 'WnvPresent',
 'dist_to_spray',
 'dist_to_ord',
 'dist_to_mdw',
 'station_name',
 'Station']

In [4]:
import imp
plotter = imp.load_source('plotter', '/Users/nichollejames/Desktop/DSI-SF-2-oboechick/week-04/plotting/knn_plotter.py')
from plotter import KNNBoundaryPlotter

In [5]:
y = train_with_spray.WnvPresent
X = train_with_spray[[col for col in train_with_spray.columns if not col in ['Date',
                                                 'Address',
                                                 'Species',
                                                 'Block',
                                                 'Street',
                                                 'Trap',
                                                 'AddressNumberAndStreet',
                                                 'Latitude',
                                                 'Longitude',
                                                 'AddressAccuracy',
                                                 'NumMosquitos',
                                                 'dist_to_spray']]].values

knn_uniform_n3 = KNeighborsClassifier(n_neighbors = 50, weights = 'uniform')
knn_uniform_n3.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')

In [6]:
train_with_spray.corr()

,Unnamed: 0,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,dist_to_spray
Unnamed: 0,1.000000,0.040425,0.147402,-0.179430,0.147362,-0.043395,0.066676,-0.290345
Block,0.040425,1.000000,0.091110,-0.090375,0.222134,-0.172388,0.004877,0.024042
Latitude,0.147402,0.091110,1.000000,-0.701795,0.444026,-0.184806,0.028697,-0.491534
Longitude,-0.179430,-0.090375,-0.701795,1.000000,-0.456775,0.036633,-0.060345,0.184654
AddressAccuracy,0.147362,0.222134,0.444026,-0.456775,1.000000,-0.248414,0.008064,-0.236795
NumMosquitos,-0.043395,-0.172388,-0.184806,0.036633,-0.248414,1.000000,0.196820,0.137007
WnvPresent,0.066676,0.004877,0.028697,-0.060345,0.008064,0.196820,1.000000,-0.047196
dist_to_spray,-0.290345,0.024042,-0.491534,0.184654,-0.236795,0.137007,-0.047196,1.000000


In [7]:
cv_indices = StratifiedKFold(y, n_folds=5)

# Function to crossvalidate accuracy of a knn model across folds
def accuracy_crossvalidator(X, y, knn, cv_indices):
    
    scores = []
    
    for train_i, test_i in cv_indices:
        
        X_train = X[train_i, :]
        X_test = X[test_i, :]

        y_train = y[train_i]
        y_test = y[test_i]

        knn.fit(X_train, y_train)
        
        acc = knn.score(X_test, y_test)
        scores.append(acc)
        
        print('Fold accuracy:', acc)
        
    print('Mean CV accuracy:', np.mean(scores))
    
accuracy_crossvalidator(X, y, knn_uniform_n3, cv_indices)

('Fold accuracy:', 0.94719314938154142)
('Fold accuracy:', 0.55354593050928125)
('Fold accuracy:', 0.94764397905759157)
('Fold accuracy:', 0.42360780580675866)
('Fold accuracy:', 0.26368396001903854)
('Mean CV accuracy:', 0.62713496495484233)


In [16]:
train_with_spray.columns.tolist()

['Unnamed: 0',
 'Date',
 'Address',
 'Species',
 'Block',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'Latitude',
 'Longitude',
 'AddressAccuracy',
 'NumMosquitos',
 'WnvPresent',
 'dist_to_spray']

In [21]:
train_with_spray.groupby(['Trap','Date'])['NumMosquitos'].sum()


Trap  Date      
T001  2007-06-26      1
      2007-07-11      1
      2007-07-18      1
      2007-08-01      4
      2007-08-07      2
      2007-08-15     10
      2007-08-21      1
      2007-08-24      2
      2007-09-12      1
      2007-10-04      2
T002  2007-05-29      2
      2007-06-05      9
      2007-06-26      6
      2007-06-29      2
      2007-07-02      5
      2007-07-11     27
      2007-07-18     13
      2007-07-27     35
      2007-08-01    179
      2007-08-07     26
      2007-08-15     76
      2007-08-21     70
      2007-08-24     62
      2007-09-04     23
      2007-09-12      8
      2007-09-18     41
      2007-09-24     74
      2007-10-04     82
      2009-05-28      2
      2009-06-02     29
                   ... 
T903  2011-06-10      1
      2011-06-17     15
      2011-06-30     95
      2011-07-11    145
      2011-07-15    174
      2011-07-25    152
      2011-07-29     85
      2011-08-05     57
      2011-08-12     65
      2011-08-19     12